Testing an Automation Listener

In [ ]:
@Operation(id = "Scripting.Listener")
function run(input, params) {

    Console.log("from js listener");
    // check input
    Console.logAsJson(input);
    Console.logAsJson(params);

    // dump the whole context
    Debug.dumpCtx();
    
    // introspection example
    Console.log("Introspection Example");    
    Debug.introspect(ctx.Event);
    
    var doc = ctx.Event.getContext().getSourceDocument();
    
    doc.setPropertyValue("dc:source","updated from listener");   
    
    if (doc.id==null) {
        Console.log("Called via Sync Listener");
    } else {
        Console.log("Called via Async Listener");
        // need to save the document
        ctx.Event.getContext().getCoreSession().saveDocument(doc);
    }
    
    
}

Do a test running the script from inside a synchronous listener

In [ ]:
@Test(id = "TestingSynchronousListener")
Harness.registerListener('aboutToCreate','Scripting.Listener',true);


var root = Repository.GetDocument(null, {
        "value" : "/"
    });

var newDoc = Document.Create(root, {
        "type" : "File",
        "name" : "newDoc",
        "properties" : {
            "dc:title" : "whathever"
        }
    });

Assert.assertTrue(newDoc.getPropertyValue("dc:source").equals("updated from listener"))
    

Do a test running the script from inside an  **a**synchronous listener

In [ ]:
@Test(id = "TestingAsynchronousListener")
Console.log("starting listener testing");

Harness.registerListener('documentCreated','Scripting.Listener',false);

var root = Repository.GetDocument(null, {
        "value" : "/"
    });

var newDoc = Document.Create(root, {
        "type" : "File",
        "name" : "newDoc",
        "properties" : {
            "dc:title" : "whathever"
        }
    });

Console.log("created test Document " + newDoc.id);

Harness.waitForAsync();


newDoc = Repository.GetDocument(null, {
        "value" : newDoc.id
    });

Assert.assertTrue(newDoc.getPropertyValue("dc:source").equals("updated from listener"))

Console.log("end of listener testing");

